In [0]:
%run "./00 - Setup Data Sets"

In [0]:
from pyspark.sql.functions import *

In [0]:
# ON <specific conditions>
users_df \
    .join(course_enrolments_df, users_df.user_id == course_enrolments_df.user_id, "inner") \
    .show()

# ON <common columns>
# common columns will occur only once in the result table
users_df \
    .join(course_enrolments_df, 'user_id', "inner") \
    .show()

# BROADCAST
broadcast(users_df) \
    .join(course_enrolments_df, 'user_id', "inner") \
    .show()

# JOIN OPTIONS
"""
> inner (default)                       INNER JOIN
> outer, full, fullouter, full_outer    FULL OUTER JOIN
> left, leftouter, left_outer           LEFT JOIN
> right, rightouter, right_outer        RIGHT JOIN
> anti, leftanti, left_anti             Reutrns leftovers from left table after filtering with right table
> semi, leftsemi, left_semi             Filters left table with right table
> cross                                 Cartesian (You can also use .crossJoin(<table>))
"""

In [0]:
# JOIN OPTIONS

"""
> inner (default)                       INNER JOIN
> outer, full, fullouter, full_outer    FULL OUTER JOIN
> left, leftouter, left_outer           LEFT JOIN
> right, rightouter, right_outer        RIGHT JOIN
> anti, leftanti, left_anti             Reutrns leftovers from left table after filtering with right table
> semi, leftsemi, left_semi             Filters left table with right table
> cross                                 Cartesian (You can also use .crossJoin(<table>))
"""

In [0]:
# EXAMPLE

# ON <specific conditions>
users_df \
    .join(course_enrolments_df, users_df.user_id == course_enrolments_df.user_id, "inner") \
    .show()

# ON <common columns>
# common columns will occur only once in the result table
users_df \
    .join(course_enrolments_df, 'user_id', "inner") \
    .show()

In [0]:
# ALIAS

# We can't use aliases in ON conditions, but we can use it in the next operations:

users_df.alias('u') \
    .join(course_enrolments_df.alias('c'), 'user_id') \
    .filter(col('c.course_id') <= lit(3)) \
    .select('c.*', 'u.user_email') \
    .show()

In [0]:
# BROADCAST JOIN

"""
In a broadcast join, the smaller DataFrame is broadcasted to all executors and kept in memory. The larger DataFrame is split and distributed across the executors. This allows for a join without shuffling any data, as the required data is already colocated on each executor.
"""

# This threshold says that if the size of the smaller DataFrame is smaller than or equal to this threshold, it will be broadcasted automatically.
spark.conf.get('spark.sql.autoBroadcastJoinThreshold')

# Broadcast example:
broadcast(users_df) \
    .join(course_enrolments_df, 'user_id', "inner") \
    .show()